In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import time
import datetime
import requests
from tiingo import TiingoClient
from yahoofinancials import YahooFinancials

from tools import *

%config Completer.use_jedi = False

### Get list of S&P 500 companies

In [2]:
sp500 = get_sp500_list()
sp500.head(3)

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)


### Download meta data information with Yahoo!

In [160]:
# get_meta_data_information
def get_meta_data_information(symbols):
    infos = []
    for i, symbol in enumerate(symbols):
        print(i+1, ':', symbol, end='\t')
        try:
            ticker = yf.Ticker(symbol)
            info = ticker.info
            infos.append(info)
        except Exception as e:
            print(e)
    return infos

sp_500_meta_data = get_meta_data_information(sp500['Symbol'].values[:5])
sp_500_meta_data = pd.DataFrame(sp_500_meta_data)
# sp_500_meta_data.to_csv("sp_500_meta_data.csv", index=False)
sp_500_meta_data.head(3)

1 : MMM	2 : ABT	3 : ABBV	4 : ABMD	5 : ACN	

,zip,sector,fullTimeEmployees,longBusinessSummary,city,phone,state,country,companyOfficers,website,...,lastCapGain,shortPercentOfFloat,sharesShortPriorMonth,impliedSharesOutstanding,category,fiveYearAverageReturn,regularMarketPrice,logo_url,address2,fax
0,55144-1000,Industrials,95000,"3M Company develops, manufactures, and markets...",St. Paul,651-733-1110,MN,United States,[],http://www.3m.com,...,None,0.0174,10277752,NaN,None,None,200.430,https://logo.clearbit.com/3m.com,NaN,NaN
1,60064-6400,Healthcare,109000,"Abbott Laboratories discovers, develops, manuf...",North Chicago,224 667 6100,IL,United States,[],http://www.abbott.com,...,None,0.0082,14598473,NaN,None,None,116.860,https://logo.clearbit.com/abbott.com,Abbott Park,NaN
2,60064-6400,Healthcare,48000,"AbbVie Inc. discovers, develops, manufactures,...",North Chicago,847 932 7900,IL,United States,[],http://www.abbvie.com,...,None,0.0087,18705085,NaN,None,None,114.335,https://logo.clearbit.com/abbvie.com,NaN,NaN


In [164]:
sp_500_meta_data[['symbol', 'marketCap','longName','sector']].sort_values(by ='marketCap', ascending=False).head(10) 

,symbol,marketCap,longName,sector
1,ABT,207639183360,Abbott Laboratories,Healthcare
2,ABBV,201940762624,AbbVie Inc.,Healthcare
4,ACN,180638154752,Accenture plc,Technology
0,MMM,116184260608,3M Company,Industrials
3,ABMD,12668157952,"Abiomed, Inc.",Healthcare


In [ ]:
useful_info = ['symbol', 'sector', 'fullTimeEmployees', 'longBusinessSummary', 
               'country', 'maxAge', 'industry', 'twoHundredDayAverage', 'trailingAnnualDividendYield', 
               'payoutRatio', 'volume24Hr', 'navPrice', 'averageDailyVolume10Day', 'totalAssets', 
               'fiftyDayAverage', 'trailingAnnualDividendRate', 'averageVolume10days', 'dividendRate', 
               'beta',  'startDate', 'priceHint', 'marketCap', 'averageVolume', 'volume', 
               'fiftyTwoWeekHigh', 'fiftyTwoWeekLow', 'dividendYield', 'exchange', 'shortName', 'longName', 
                'market', 'beta3Year', '52WeekChange', 'morningStarRiskRating', 'forwardEps', 
               'revenueQuarterlyGrowth', 'sharesOutstanding', 'fundInceptionDate', 'annualReportExpenseRatio', 
               'bookValue', 'sharesShort', 'sharesPercentSharesOut', 'fundFamily', 'lastFiscalYearEnd', 
               'heldPercentInstitutions', 'netIncomeToCommon', 'trailingEps', 
               'lastDividendValue', 'SandP52WeekChange', 'priceToBook', 
               'heldPercentInsiders', 'nextFiscalYearEnd', 'mostRecentQuarter', 'shortRatio', 
               'sharesShortPreviousMonthDate', 'floatShares', 'enterpriseValue', 'threeYearAverageReturn', 
               'lastSplitDate', 'lastSplitFactor', 'legalType', 'lastDividendDate', 
               'morningStarOverallRating', 'earningsQuarterlyGrowth', 'dateShortInterest', 'pegRatio',
               'lastCapGain', 'shortPercentOfFloat', 'sharesShortPriorMonth', 
               'impliedSharesOutstanding', 'category', 'fiveYearAverageReturn', 
               'regularMarketPrice']

# Try different libraries

In [101]:
## Define variables
ticker = 'AAPL'
start = '2020-08-27' # datetime.datetime(year,1,1)
end = '2020-9-1'  # datetime.datetime(year,1,5)

## 1. Tiingo
**Source:** https://pypi.org/project/tiingo/

In [91]:
config = {}

# To reuse the same HTTP Session across API calls (and have better performance), include a session key.
config['session'] = True

# If you don't have your API key as an environment variable,
# pass it in via a configuration dictionary.
tingo_api_key = "2b42ef195306a06576049bb360841c5e776addc6"
config['api_key'] = tingo_api_key

# Initialize
client = TiingoClient(config)

#### What tickers are available? 
Not CAD stocks :(

In [92]:
# Check what tickers are available, as well as metadata about each ticker
# including supported currency, exchange, and available start/end dates.
available_tiingo_tickers = client.list_stock_tickers()

tickers = pd.DataFrame(available_tiingo_tickers)
tickers[(tickers['priceCurrency']=='USD')] ['exchange'].unique()
tickers[(tickers['exchange']=='NYSE')].head(4)

,ticker,exchange,assetType,priceCurrency,startDate,endDate
77,0001753539,NYSE,Stock,USD,,
4567,A,NYSE,Stock,USD,1999-11-18,2021-06-18
4568,AA,NYSE,Stock,USD,2016-11-01,2021-06-18
4569,AA-W,NYSE,Stock,USD,2016-10-18,2016-11-08
4583,AAC,NYSE,Stock,USD,2021-03-25,2021-06-18


#### Get ticker information

In [94]:
# Get Ticker
ticker_metadata = client.get_ticker_metadata(ticker)
ticker_metadata

{'name': 'Apple Inc',
 'description': "Apple Inc. (Apple) designs, manufactures and markets mobile communication and media devices, personal computers, and portable digital music players, and a variety of related software, services, peripherals, networking solutions, and third-party digital content and applications. The Company's products and services include iPhone, iPad, Mac, iPod, Apple TV, a portfolio of consumer and professional software applications, the iOS and OS X operating systems, iCloud, and a variety of accessory, service and support offerings. The Company also delivers digital content and applications through the iTunes Store, App StoreSM, iBookstoreSM, and Mac App Store. The Company distributes its products worldwide through its retail stores, online stores, and direct sales force, as well as through third-party cellular network carriers, wholesalers, retailers, and value-added resellers. In February 2012, the Company acquired app-search engine Chomp.",
 'ticker': 'AAPL'

In [102]:
# Get historical GOOGL prices from August 2017 as JSON, sampled daily
historical_prices = client.get_dataframe(ticker,
                                            startDate=start, endDate=end,
                                            frequency='daily')  #5min, ... ,1hour, daily
# for frequency less than daily, it does not provide volume, etc..
historical_prices

,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
date,,,,,,,,,,,,
2020-08-27 00:00:00+00:00,500.04,509.94,495.33,508.57,38888096,124.397493,126.860367,123.225763,126.519545,155552384,0.0,1.0
2020-08-28 00:00:00+00:00,499.23,505.77,498.31,504.05,46907479,124.195985,125.822974,123.967112,125.395081,187629916,0.0,1.0
2020-08-31 00:00:00+00:00,129.04,131.00,126.00,127.58,223505733,128.407748,130.358144,125.382643,126.954901,223505733,0.0,4.0
2020-09-01 00:00:00+00:00,134.18,134.80,130.53,132.76,152470142,133.522563,134.139525,129.890447,132.109521,152470142,0.0,1.0


In [98]:
# Get definitions for fields available in the fundamentals-api, ticker is
# optional
definitions = client.get_fundamentals_definitions(ticker)
pd.DataFrame(definitions).head(10)

,dataCode,name,description,statementType,units
0,rps,Revenue Per Share,Revenue per share,overview,$
1,roa,Return on Assets ROA,Net Income/Total Assets,overview,%
2,assetTurnover,Asset Turnover,Revenue over assets,overview,None
3,bookVal,Book Value,"Book value of the share, assets - liabilities",overview,$
4,bvps,Book Value Per Share,Book Value per each share,overview,$
5,revenue,Revenue,Revenue,incomeStatement,$
6,epsDil,Earnings Per Share Diluted,EPS for diluted shares,incomeStatement,$
7,netinc,Net Income,Net income,incomeStatement,$
8,profitMargin,Profit Margin,This field is marked for DEPRECATION. Please u...,overview,%
9,revenueQoQ,Revenue QoQ Growth,Revenue Quarter-over-Quarter Growth rate,overview,%


In [103]:
# Get fundamentals which require daily-updated (like marketCap). A start-
# and end-date can be passed. If omited, will get all available data.
fundamentals_daily = client.get_fundamentals_daily(ticker,
                                        startDate=start,
                                        endDate=end)
pd.DataFrame(fundamentals_daily)

,date,marketCap,enterpriseVal,peRatio,pbRatio,trailingPEG1Y
0,2020-08-27T00:00:00.000Z,2.137988e+12,2.217328e+12,37.744565,29.984517,2.035246
1,2020-08-28T00:00:00.000Z,2.134525e+12,2.213865e+12,37.683424,29.935946,2.031949
2,2020-08-31T00:00:00.000Z,2.206911e+12,2.286251e+12,38.961353,30.951141,2.100857
3,2020-09-01T00:00:00.000Z,2.294818e+12,2.374158e+12,40.513285,32.184005,2.184540


In [92]:
(pd.DataFrame(fundamentals_daily)['marketCap'].values / historical_prices['close'].values)

array([5.62791233e+08, 5.62791233e+08, 5.62791233e+08, 5.62791233e+08])

In [104]:
client.get_fundamentals_statements(ticker,
                                                       startDate='2020-01-01',
                                                      #   endDate='2020-12-31',
                                                         asReported=True)

[{'year': 2021,
  'quarter': 2,
  'statementData': {'cashFlow': [{'dataCode': 'ncff', 'value': -11326000000.0},
    {'dataCode': 'ncfx', 'value': 0.0},
    {'dataCode': 'investmentsAcqDisposals', 'value': -7895000000.0},
    {'dataCode': 'ncf', 'value': 2287000000.0},
    {'dataCode': 'ncfi', 'value': -10368000000.0},
    {'dataCode': 'payDiv', 'value': -3447000000.0},
    {'dataCode': 'sbcomp', 'value': 1981000000.0},
    {'dataCode': 'issrepayEquity', 'value': -17987000000.0},
    {'dataCode': 'businessAcqDisposals', 'value': 0.0},
    {'dataCode': 'capex', 'value': -2269000000.0},
    {'dataCode': 'ncfo', 'value': 23981000000.0},
    {'dataCode': 'issrepayDebt', 'value': 10423000000.0},
    {'dataCode': 'freeCashFlow', 'value': 21712000000.0},
    {'dataCode': 'depamor', 'value': 2797000000.0}],
   'incomeStatement': [{'dataCode': 'consolidatedIncome',
     'value': 23630000000.0},
    {'dataCode': 'grossProfit', 'value': 38079000000.0},
    {'dataCode': 'shareswa', 'value': 1675347

In [98]:
# ticker_price['date'] = ticker_price.index.to_series().dt.date
# ticker_price['time'] = ticker_price.index.to_series().dt.time
historical_prices['date-time'] = historical_prices.index.to_series()
historical_prices.head(5) #reset_index().

,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor,date-time
date,,,,,,,,,,,,,
2020-01-02 00:00:00+00:00,333.32,333.35,327.70,328.55,4543965,331.333853,331.363674,325.747340,326.592275,4543965,0.0,1.0,2020-01-02 00:00:00+00:00
2020-01-03 00:00:00+00:00,332.76,334.89,330.30,330.63,3875835,330.777189,332.894497,328.331848,328.659881,3875835,0.0,1.0,2020-01-03 00:00:00+00:00
2020-01-06 00:00:00+00:00,333.74,334.86,327.88,329.30,5353250,331.751350,332.864676,325.926268,327.337806,5353250,0.0,1.0,2020-01-06 00:00:00+00:00
2020-01-07 00:00:00+00:00,337.28,344.19,330.71,334.26,9901768,335.270256,342.139082,328.739405,332.268251,9901768,0.0,1.0,2020-01-07 00:00:00+00:00


In [101]:
diff_in_seconds = historical_prices[['date-time']].diff()#.values.reshape(-1)/60000000000 
(diff_in_seconds)

,date-time
date,
2020-01-02 00:00:00+00:00,NaT
2020-01-03 00:00:00+00:00,1 days
2020-01-06 00:00:00+00:00,3 days
2020-01-07 00:00:00+00:00,1 days


In [117]:
type(historical_prices.iloc[0, -1]) 
import datetime
datetimeFormat = "%H:%M:%S"
t0 = historical_prices.iloc[0, -1]
t1 = historical_prices.iloc[1, -1]
datetime.datetime.combine(datetime.date.today(), t1) - datetime.datetime.combine(datetime.date.today(), t0)

# datetime.strptime(t1, datetimeFormat) - datetime.strptime(t0, datetimeFormat)

TypeError: combine() argument 2 must be datetime.time, not Timestamp

In [118]:
import datetime
datetimeFormat = '%H:%M:%S.%f'
date1 = '10:01:28.585'
date2 = '09:56:28.067'
diff = datetime.datetime.strptime(date1, datetimeFormat)\
    - datetime.datetime.strptime(date2, datetimeFormat)
 
print("Difference:", diff)
print("Days:", diff.days)
print("Microseconds:", diff.microseconds)
print("Seconds:", diff.seconds)

Difference: 0:05:00.518000
Days: 0
Microseconds: 518000
Seconds: 300


## 2. pandas_datareader
https://readthedocs.org/projects/pandas-datareader/downloads/pdf/latest/

In [137]:
import os
import pandas_datareader as pdr

df = pdr.get_data_tiingo('GOOG', api_key=tingo_api_key, start="2020-01-01",  end="2020-01-07", freq='hourly')
                                            
df.head()

close     high      low     open   volume  \
symbol date                                                                     
GOOG   2020-01-02 00:00:00+00:00  1367.37  1368.14  1341.55  1341.55  1406731   
       2020-01-03 00:00:00+00:00  1360.66  1372.50  1345.54  1347.86  1187006   
       2020-01-06 00:00:00+00:00  1394.21  1396.50  1350.00  1350.00  1733149   
       2020-01-07 00:00:00+00:00  1393.34  1402.99  1390.38  1397.94  1511693   

                                  adjClose  adjHigh   adjLow  adjOpen  \
symbol date                                                             
GOOG   2020-01-02 00:00:00+00:00   1367.37  1368.14  1341.55  1341.55   
       2020-01-03 00:00:00+00:00   1360.66  1372.50  1345.54  1347.86   
       2020-01-06 00:00:00+00:00   1394.21  1396.50  1350.00  1350.00   
       2020-01-07 00:00:00+00:00   1393.34  1402.99  1390.38  1397.94   

                                  adjVolume  divCash  splitFactor  
symbol date                                                        
GOOG   2020-01-02 00:00:00+00:00    1406731      0.0          1.0  
       2020-01-03 00:00:00+00:00    1187006      0.0          1.0  
       2020-01-06 00:00:00+00:00    1733149      0.0          1.0  
       2020-01-07 00:00:00+00:00    1511693      0.0          1.0

In [151]:
a = pdr.tiingo.TiingoQuoteReader('GOOG', api_key=tingo_api_key, start="2020-01-01",  end="2020-01-07", freq='5mins')
a.read()

,,adjClose,adjHigh,adjLow,adjOpen,adjVolume,close,divCash,high,low,open,splitFactor,volume
symbol,date,,,,,,,,,,,,
GOOG,2021-05-25 00:00:00+00:00,2409.07,2432.89,2402.99,2420.0,938644,2409.07,0.0,2432.89,2402.99,2420.0,1.0,938644


## 3. Alpha vintage
Can't determine start and end date

Very good for EOD data for even TSX

In [1]:
from alpha_vantage.timeseries import TimeSeries 
 
ALPHA_VANTAGE_API_KEY = 'I3Z5IQNTGCQ7IED4'  # J4RADOTW5DBN344U

# Initialize the TimeSeries class with key and output format 
ts = TimeSeries(key=ALPHA_VANTAGE_API_KEY, output_format='pandas') 
 
# # Get json object with the intraday data and information of the data 
# intraday_data, data_info = ts.get_intraday( 'GOOGL')  # outputsize='full', interval='1min'

#### Get full daily adjusted data

In [2]:
data, meta_data = ts.get_daily_adjusted(symbol='AC.TO', outputsize='full')
data

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2021-06-16,28.24,28.32,27.66,28.10,28.10,2448470.0,0.0,1.0
2021-06-15,28.37,28.61,28.13,28.25,28.25,2768100.0,0.0,1.0
2021-06-14,28.70,28.76,28.10,28.34,28.34,3515800.0,0.0,1.0
2021-06-11,28.56,29.16,28.44,28.55,28.55,5884100.0,0.0,1.0
2021-06-10,28.80,28.88,28.20,28.41,28.41,3020000.0,0.0,1.0
...,...,...,...,...,...,...,...,...
2006-11-23,19.29,19.49,19.01,19.45,19.45,423204.0,0.0,1.0
2006-11-22,17.75,19.13,17.75,19.10,19.10,616310.0,0.0,1.0
2006-11-21,18.75,18.75,17.50,17.86,17.86,1943962.0,0.0,1.0


#### Time_Series_Intraday
This API returns intraday time series of the equity specified, covering extended trading hours where applicable (e.g., 4:00am to 8:00pm Eastern Time for the US market). The intraday data is derived from the Securities Information Processor (SIP) market-aggregated data. You can query both raw (as-traded) and split/dividend-adjusted intraday data from this endpoint.


This API returns the most recent 1-2 months of intraday data and is best suited for short-term/medium-term charting and trading strategy development. If you are targeting a deeper intraday history, please use the Extended Intraday API.

In [ ]:
# Get json object with the intraday data and another with  the call's metadata
data, meta_data = ts.get_intraday('AAPL', interval='15min', outputsize='full')
pd.DataFrame(data).head(4)

#### Intraday (Extended History)

This API returns historical intraday time series for the trailing 2 years, covering over 2 million data points per ticker. The intraday data is derived from the Securities Information Processor (SIP) market-aggregated data. You can query both raw (as-traded) and split/dividend-adjusted intraday data from this endpoint. Common use cases for this API include data visualization, trading simulation/backtesting, and machine learning and deep learning applications with a longer horizon.

***Did NOT work***

## 4. Marketstack
Not accurate, many missings

In [6]:
import requests

access_key = '5a4dc7d293c8c4787b34a37359849192'
params = {
  'access_key': '5a4dc7d293c8c4787b34a37359849192',
    'date_from': '2020-09-01' }

api_result = requests.get('http://api.marketstack.com/v1/tickers/aapl/eod', params)
api_response = api_result.json()

In [9]:
pd.DataFrame(api_result.json()['data']['eod']).head(4)

,open,high,low,close,volume,adj_high,adj_low,adj_close,adj_open,adj_volume,split_factor,symbol,exchange,date
0,130.71,131.51,130.250,130.46,108953309.0,NaN,NaN,130.46,NaN,NaN,1.0,AAPL,XNAS,2021-06-18T00:00:00+0000
1,129.80,132.55,129.650,131.79,96230975.0,NaN,NaN,131.79,NaN,NaN,1.0,AAPL,XNAS,2021-06-17T00:00:00+0000
2,130.37,130.89,128.461,130.15,91815026.0,NaN,NaN,130.15,NaN,NaN,1.0,AAPL,XNAS,2021-06-16T00:00:00+0000
3,129.94,130.59,129.390,129.64,62345416.0,NaN,NaN,129.64,NaN,NaN,1.0,AAPL,XNAS,2021-06-15T00:00:00+0000


## 5. YahooFinancials

In [72]:
ticker = 'AAPL'
yahoo_financials = YahooFinancials(ticker)

#### get_financial_stmts

In [73]:
balance_sheet_data_qt = yahoo_financials.get_financial_stmts('quarterly', 'balance')

In [74]:
n_balance_sheets = len(balance_sheet_data_qt['balanceSheetHistoryQuarterly'][ticker])
ind = [list(balance_sheet_data_qt['balanceSheetHistoryQuarterly'][ticker][i].keys())[0] for i in range(n_balance_sheets)]
print(ind)

vals = [list(balance_sheet_data_qt['balanceSheetHistoryQuarterly'][ticker][i].values())[0] for i in range(n_balance_sheets)]
pd.DataFrame(vals, index=ind)

['2021-03-27', '2020-12-26', '2020-09-26', '2020-06-27']


,totalLiab,totalStockholderEquity,otherCurrentLiab,totalAssets,commonStock,otherCurrentAssets,retainedEarnings,otherLiab,treasuryStock,otherAssets,...,otherStockholderEquity,propertyPlantEquipment,totalCurrentAssets,longTermInvestments,netTangibleAssets,shortTermInvestments,netReceivables,longTermDebt,inventory,accountsPayable
2021-03-27,267980000000,69178000000,53255000000,337158000000,54203000000,13376000000,15261000000,39853000000,-286000000,43339000000,...,-286000000,37815000000,121465000000,134539000000,69178000000,31368000000,33036000000,108642000000,5219000000,40127000000
2020-12-26,287830000000,66224000000,55899000000,354054000000,51744000000,13687000000,14301000000,43042000000,179000000,43270000000,...,179000000,37933000000,154106000000,118745000000,66224000000,40816000000,58620000000,99281000000,4973000000,63846000000
2020-09-26,258549000000,65339000000,47867000000,323888000000,50779000000,11264000000,14966000000,46108000000,-406000000,33952000000,...,-406000000,45336000000,143713000000,100887000000,65339000000,52927000000,37445000000,98667000000,4061000000,42296000000
2020-06-27,245062000000,72282000000,39945000000,317344000000,48696000000,10987000000,24136000000,47606000000,-550000000,32836000000,...,-550000000,43851000000,140065000000,100592000000,72282000000,59642000000,32075000000,94048000000,3978000000,35325000000


In [105]:
yahoo_financials.get_market_cap()

2207769427968

In [84]:
frequency = 'quarterly' # frequency can be either 'annual' or 'quarterly'.
statement_type = ['income','balance', 'cash'] # statement_type can be 'income', 'balance', 'cash' or a list of several.
statement_data = yahoo_financials.get_financial_stmts(frequency, statement_type)

statement_data_dfs = []
for k in statement_data.keys():
    
    n = len(statement_data[k][ticker])
    ind = [list(statement_data[k][ticker][i].keys())[0] for i in range(n)]
    print(ind)
    vals = [list(statement_data[k][ticker][i].values())[0] for i in range(n)]
    statement_data_dfs.append(pd.DataFrame(vals, index=ind))

['2021-03-27', '2020-12-26', '2020-09-26', '2020-06-27']
['2021-03-27', '2020-12-26', '2020-09-26', '2020-06-27']
['2021-03-27', '2020-12-26', '2020-09-26', '2020-06-27']


In [86]:
(statement_data_dfs[1]).pct_change()

,totalLiab,totalStockholderEquity,otherCurrentLiab,totalAssets,commonStock,otherCurrentAssets,retainedEarnings,otherLiab,treasuryStock,otherAssets,...,otherStockholderEquity,propertyPlantEquipment,totalCurrentAssets,longTermInvestments,netTangibleAssets,shortTermInvestments,netReceivables,longTermDebt,inventory,accountsPayable
2021-03-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-26,0.074073,-0.042701,0.049648,0.050113,-0.045366,0.023251,-0.062905,0.080019,-1.625874,-0.001592,...,-1.625874,0.003120,0.268728,-0.117393,-0.042701,0.301199,0.774428,-0.086164,-0.047135,0.591098
2020-09-26,-0.101730,-0.013364,-0.143688,-0.085202,-0.018650,-0.177029,0.046500,0.071233,-3.268156,-0.215346,...,-3.268156,0.195160,-0.067441,-0.150389,-0.013364,0.296722,-0.361225,-0.006184,-0.183390,-0.337531
2020-06-27,-0.052164,0.106261,-0.165500,-0.020205,-0.041021,-0.024592,0.612722,0.032489,0.354680,-0.032870,...,0.354680,-0.032755,-0.025384,-0.002924,0.106261,0.126873,-0.143410,-0.046814,-0.020438,-0.164815


In [87]:
earnings_data = yahoo_financials.get_stock_earnings_data()
earnings_data

{'AAPL': {'earningsData': {'quarterly': [{'date': '2Q2020',
     'actual': 0.64,
     'estimate': 0.51},
    {'date': '3Q2020', 'actual': 0.73, 'estimate': 0.7},
    {'date': '4Q2020', 'actual': 1.68, 'estimate': 1.41},
    {'date': '1Q2021', 'actual': 1.4, 'estimate': 0.99}],
   'currentQuarterEstimate': 0.99,
   'currentQuarterEstimateDate': '2Q',
   'currentQuarterEstimateYear': 2021,
   'earningsDate': [{'raw': 1627430400, 'fmt': '2021-07-28'},
    {'raw': 1627862400, 'fmt': '2021-08-02'}]},
  'financialsData': {'yearly': [{'date': 2017,
     'revenue': 229234000000,
     'earnings': 48351000000},
    {'date': 2018, 'revenue': 265595000000, 'earnings': 59531000000},
    {'date': 2019, 'revenue': 260174000000, 'earnings': 55256000000},
    {'date': 2020, 'revenue': 274515000000, 'earnings': 57411000000}],
   'quarterly': [{'date': '2Q2020',
     'revenue': 59685000000,
     'earnings': 11253000000},
    {'date': '3Q2020', 'revenue': 64698000000, 'earnings': 12673000000},
    {'date'

In [88]:
yahoo_financials.get_net_income()

57411000000

In [ ]:
historical_stock_prices = yahoo_financials.get_historical_price_data('2018-09-15', '2019-09-15', 'weekly')